In [42]:
import requests
import json

In [43]:
import mysql.connector

In [44]:
cnx = mysql.connector.connect(
    host = "flatiron.cimm2ummmfkj.us-east-1.rds.amazonaws.com",
    user = "avivamaz",
    passwd = "Ghustav1!")

In [45]:
cursor = cnx.cursor()

In [46]:
cnx.database = 'yelp'

In [47]:
def api_call(url_params,headers, url):
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()
    business = data['businesses']
    return(business) 

In [48]:
def parse_data(data):
    list_businessData = []
    for business in data:
        try:
            biz_tuple = (business['id'],business['name'], ' '.join(business['location']['display_address']),
                    business['rating'], len(business['price']), business['review_count'])
            list_businessData.append(biz_tuple)
        except:
            biz_tuple = (business['id'],business['name'], ' '.join(business['location']['display_address']),
                    business['rating'], 0, business['review_count'])
            list_businessData.append(biz_tuple)
    return list_businessData

In [49]:
#why have to put cnx and cursor into here
def Data_into_table(cnx,cursor,parsed_data):
    insert_stm = """INSERT INTO businesses (business_id, name, address, rating, price, review_count) VALUES (%s, %s, %s, %s, %s, %s)"""
    cursor.executemany(insert_stm, parsed_data)
    cnx.commit()
    print('Check the workbench')

In [50]:
cur = 0
url = 'https://api.yelp.com/v3/businesses/search'
client_id = 'O-E5Od4R_LRN58GCv0_4xQ'

api_key = 'bzeWISo1x-EG8CXGaynhcxkV___l10Fmf4ThmxYaFuIC-xRL3zGM_tS-1UZGBWqv5oCK5f0hhaZLsOufaUrQeWPXX20fitWPuuqW3HLI_2uiYuOzXS-hq2o5OWXpXXYx'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

term = 'bagel'
location = 'East Village'
radius = 5000
limit = 50


offset = cur

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'radius': radius,
                'limit': limit,
                'offset': offset
            }

In [51]:
while cur < 1000:
        business=api_call(url_params,headers, url)
        parsed_data=parse_data(business)
        Data_into_table(cnx, cursor, parsed_data)
        cur += 50
        url_params['offset'] = cur